# Faster R-CNN

Import the necessary libraries.

In [ ]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
from torch.utils.data import Dataset

import torchvision.transforms as T
import json 


Declare the possible classes of the dataset

In [4]:
CLASS_NAMES = {
  1: "Stop",
  2: "No Entry (One Way)",
  3: "Pedestrian Crossing",
  4: "Roundabout Ahead",
  5: "No Through Road (T-Sign)",
  6: "Blind-Spot Mirror (Convex)"
}

In [10]:
class SignsDataset(Dataset):
    def __init__(self, root, annFile, transforms=None):
        self.root = root
        self.transforms = transforms

        with open(annFile) as f:
            data = json.load(f)

        self.images = data["images"]
        self.annotations = data["annotations"]

        # map image to annotations
        self.imgToAnns = {img["id"]: [] for img in self.images}
        for ann in self.annotations:
            self.imgToAnns[ann["image_id"]].append(ann)

    def __getitem__(self, idx):
        img_info = self.images[idx]
        image_id = img_info["id"]

        img_path = os.path.join(self.root, img_info["file_name"])
        img = Image.open(img_path).convert("RGB")

        anns = self.imgToAnns[image_id]

        boxes = []
        labels = []

        for ann in anns:
            x, y, w, h = ann["bbox"]
            boxes.append([x, y, w, h])
            labels.append(ann["category_id"])

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([image_id])
        }

        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.images)

In [12]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())

    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))

    return T.Compose(transforms)


In [13]:
train_dataset = SignsDataset(
    root="data/images",
    annFile="data/instances_signs.json",
    transforms=get_transform(train=True)
)

val_dataset = SignsDataset(
    root="data/images",
    annFile="dataset/instances_signs.json",
    transforms=get_transform(train=False)
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/instances_signs.json'

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    weights="DEFAULT"
)

infeat = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(infeat, len(CLASS_NAMES) + 1)  # +1 for background

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for imgs, targets in train_loader:
        imgs = [img.to(device) for img in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(imgs, targets)
        loss = sum(loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), "models/fasterrcnn_signs.pth")